In [2]:
import code

import numpy as np
import gurobipy as gp

model = code.model.Model(n = 1, k_half = 3, k_upper = 9)

model.set_objective([
    (1, [0]),
    (1, [1]),
    (-3, [3]),
    (2, [4]),
])

model.add_cone(
    'boundA', 
    localizer = [
        (1, [0]),
        (-1, [2])
    ],
    inequality = True,
)

model.optimize(100)

model.vars_x

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i7-6700HQ CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads



GurobiError: Model too large for size-limited license; visit https://www.gurobi.com/free-trial for a full license